In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🔍 Анализ аналитики сайта Viktorija Autokool\n",
    "\n",
    "## Цель анализа:\n",
    "- Анализ поведения пользователей\n",
    "- Пути навигации (funnel analysis)\n",
    "- Время на странице\n",
    "- Конверсия до checkout\n",
    "- Визуализация данных\n",
    "- Инсайты и рекомендации по улучшению UX"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Импорт необходимых библиотек\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "from datetime import datetime\n",
    "import re\n",
    "\n",
    "# Настройка отображения\n",
    "plt.style.use('seaborn-v0_8')\n",
    "pd.set_option('display.max_columns', None)\n",
    "pd.set_option('display.width', None)\n",
    "\n",
    "print(\"✅ Библиотеки загружены\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Загрузка и подготовка данных"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Загрузка данных\n",
    "df = pd.read_csv('Analytics/inspectlet-results-export.csv')\n",
    "print(f\"📈 Загружено {len(df)} записей\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Информация о данных\n",
    "print(\"📋 Информация о датасете:\")\n",
    "print(df.info())\n",
    "print(\"\\n🔍 Первые несколько строк:\")\n",
    "print(df.head())\n",
    "print(\"\\n📊 Статистика:\")\n",
    "print(df.describe())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧹 Очистка и подготовка данных"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Функция для извлечения времени сессии в минутах\n",
    "def extract_session_time(time_str):\n",
    "    if pd.isna(time_str):\n",
    "        return 0\n",
    "    \n",
    "    # Извлечение времени из строки\n",
    "    time_parts = time_str.split()\n",
    "    total_minutes = 0\n",
    "    \n",
    "    for i, part in enumerate(time_parts):\n",
    "        if part.isdigit():\n",
    "            if i + 1 < len(time_parts):\n",
    "                next_part = time_parts[i + 1]\n",
    "                if 'min' in next_part:\n",
    "                    total_minutes += int(part)\n",
    "                elif 'sec' in next_part:\n",
    "                    total_minutes += int(part) / 60\n",
    "    \n",
    "    return total_minutes\n",
    "\n",
    "# Функция для извлечения количества страниц\n",
    "def extract_pages_count(pages_str):\n",
    "    if pd.isna(pages_str):\n",
    "        return 0\n",
    "    \n",
    "    # Извлечение числа страниц\n",
    "    match = re.search(r'(\\d+)\\s+page', pages_str)\n",
    "    if match:\n",
    "        return int(match.group(1))\n",
    "    return 0\n",
    "\n",
    "# Функция для извлечения даты и времени\n",
    "def extract_datetime(date_str):\n",
    "    if pd.isna(date_str):\n",
    "        return pd.NaT\n",
    "    \n",
    "    try:\n",
    "        # Парсинг даты в формате \"Sep 3 1031 am\"\n",
    "        # Добавляем год 2024 для корректного парсинга\n",
    "        date_with_year = f\"{date_str} 2024\"\n",
    "        parsed_date = pd.to_datetime(date_with_year, format='%b %d %I%M %p %Y')\n",
    "        return parsed_date\n",
    "    except:\n",
    "        return pd.NaT\n",
    "\n",
    "# Применение функций очистки\n",
    "df_clean = df.copy()\n",
    "df_clean['session_time_minutes'] = df_clean['Session Length'].apply(extract_session_time)\n",
    "df_clean['pages_count'] = df_clean['Pages Visited'].apply(extract_pages_count)\n",
    "df_clean['session_datetime'] = df_clean['Session Start'].apply(extract_datetime)\n",
    "df_clean['session_date'] = df_clean['session_datetime'].dt.date\n",
    "df_clean['session_hour'] = df_clean['session_datetime'].dt.hour\n",
    "\n",
    "# Очистка источника трафика\n",
    "df_clean['clean_referrer'] = df_clean['Referrer'].fillna('Direct')\n",
    "df_clean['is_direct'] = df_clean['clean_referrer'].str.contains('Direct', case=False, na=False)\n",
    "df_clean['is_google'] = df_clean['clean_referrer'].str.contains('google', case=False, na=False)\n",
    "df_clean['is_internal'] = df_clean['clean_referrer'].str.contains('viktorijaautokool', case=False, na=False)\n",
    "\n",
    "# Анализ путей навигации\n",
    "df_clean['has_checkout'] = df_clean['Pages Visited'].str.contains('checkout', case=False, na=False)\n",
    "df_clean['has_packages'] = df_clean['Pages Visited'].str.contains('packages', case=False, na=False)\n",
    "df_clean['has_services'] = df_clean['Pages Visited'].str.contains('services', case=False, na=False)\n",
    "\n",
    "print(\"✅ Данные очищены и подготовлены\")\n",
    "df_clean.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Основные метрики и статистика"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Основные метрики\n",
    "print(\"🎯 ОСНОВНЫЕ МЕТРИКИ САЙТА\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "total_sessions = len(df_clean)\n",
    "avg_session_time = df_clean['session_time_minutes'].mean()\n",
    "avg_pages = df_clean['pages_count'].mean()\n",
    "bounce_rate = (df_clean['pages_count'] == 1).mean() * 100\n",
    "checkout_conversion = df_clean['has_checkout'].mean() * 100\n",
    "\n",
    "print(f\"📊 Общее количество сессий: {total_sessions}\")\n",
    "print(f\"⏱️  Средняя длительность сессии: {avg_session_time:.2f} минут\")\n",
    "print(f\"📄 Среднее количество страниц: {avg_pages:.2f}\")\n",
    "print(f\"🔄 Bounce Rate: {bounce_rate:.1f}%\")\n",
    "print(f\"💰 Конверсия до checkout: {checkout_conversion:.1f}%\")\n",
    "\n",
    "# Дополнительная статистика\n",
    "print(\"\\n📈 ДОПОЛНИТЕЛЬНАЯ СТАТИСТИКА\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "print(f\"🕐 Медианное время сессии: {df_clean['session_time_minutes'].median():.2f} минут\")\n",
    "print(f\"📊 Медианное количество страниц: {df_clean['pages_count'].median():.0f}\")\n",
    "print(f\"⚡ Максимальное время сессии: {df_clean['session_time_minutes'].max():.2f} минут\")\n",
    "print(f\"📚 Максимальное количество страниц: {df_clean['pages_count'].max():.0f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎯 Анализ поведения пользователей"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Сегментация пользователей по длительности сессии\n",
    "print(\"👥 СЕГМЕНТАЦИЯ ПОЛЬЗОВАТЕЛЕЙ ПО ДЛИТЕЛЬНОСТИ СЕССИИ\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "# Создание сегментов\n",
    "def categorize_session_time(minutes):\n",
    "    if minutes < 1:\n",
    "        return 'Быстрые (< 1 мин)'\n",
    "    elif minutes < 5:\n",
    "        return 'Короткие (1-5 мин)'\n",
    "    elif minutes < 15:\n",
    "        return 'Средние (5-15 мин)'\n",
    "    else:\n",
    "        return 'Долгие (> 15 мин)'\n",
    "\n",
    "df_clean['session_category'] = df_clean['session_time_minutes'].apply(categorize_session_time)\n",
    "\n",
    "# Анализ сегментов\n",
    "session_segments = df_clean.groupby('session_category').agg({\n",
    "    'Display Name': 'count',\n",
    "    'session_time_minutes': ['mean', 'median'],\n",
    "    'pages_count': ['mean', 'median'],\n",
    "    'has_checkout': 'mean'\n",
    "}).round(3)\n",
    "\n",
    "session_segments.columns = ['Количество сессий', 'Среднее время', 'Медианное время', \n",
    "                           'Среднее кол-во страниц', 'Медианное кол-во страниц', 'Конверсия checkout']\n",
    "\n",
    "print(session_segments)\n",
    "\n",
    "# Процентное распределение\n",
    "print(\"\\n📊 ПРОЦЕНТНОЕ РАСПРЕДЕЛЕНИЕ СЕГМЕНТОВ:\")\n",
    "segment_percentages = df_clean['session_category'].value_counts(normalize=True) * 100\n",
    "for segment, percentage in segment_percentages.items():\n",
    "    print(f\"{segment}: {percentage:.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🛤️ Пути навигации (Funnel Analysis)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Анализ воронки конверсии\n",
    "print(\"🛤️  ВОРОНКА КОНВЕРСИИ\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "# Основные этапы воронки\n",
    "funnel_steps = {\n",
    "    'Главная страница': len(df_clean),\n",
    "    'Услуги/Packages': df_clean['has_packages'].sum(),\n",
    "    'Checkout': df_clean['has_checkout'].sum()\n",
    "}\n",
    "\n",
    "print(\"📊 Количество пользователей на каждом этапе:\")\n",
    "for step, count in funnel_steps.items():\n",
    "    print(f\"{step}: {count}\")\n",
    "\n",
    "print(\"\\n📈 Конверсия между этапами:\")\n",
    "prev_count = None\n",
    "for step, count in funnel_steps.items():\n",
    "    if prev_count is not None:\n",
    "        conversion = (count / prev_count) * 100\n",
    "        print(f\"{step}: {conversion:.1f}% (из предыдущего этапа)\")\n",
    "    else:\n",
    "        print(f\"{step}: 100% (базовый уровень)\")\n",
    "    prev_count = count\n",
    "\n",
    "# Общая конверсия\n",
    "overall_conversion = (funnel_steps['Checkout'] / funnel_steps['Главная страница']) * 100\n",
    "print(f\"\\n🎯 Общая конверсия в checkout: {overall_conversion:.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ⏱️ Анализ времени на странице"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Анализ времени на странице\n",
    "print(\"⏱️  АНАЛИЗ ВРЕМЕНИ НА СТРАНИЦЕ\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "# Распределение времени сессий\n",
    "print(\"📊 РАСПРЕДЕЛЕНИЕ ВРЕМЕНИ СЕССИЙ:\")\n",
    "time_stats = df_clean['session_time_minutes'].describe()\n",
    "print(f\"Мин: {time_stats['min']:.2f} мин\")\n",
    "print(f\"25%: {time_stats['25%']:.2f} мин\")\n",
    "print(f\"Медиана: {time_stats['50%']:.2f} мин\")\n",
    "print(f\"75%: {time_stats['75%']:.2f} мин\")\n",
    "print(f\"Макс: {time_stats['max']:.2f} мин\")\n",
    "\n",
    "# Время на страницу (приблизительно)\n",
    "df_clean['time_per_page'] = df_clean['session_time_minutes'] / df_clean['pages_count']\n",
    "df_clean['time_per_page'] = df_clean['time_per_page'].replace([np.inf, -np.inf], np.nan)\n",
    "\n",
    "print(f\"\\n📄 СРЕДНЕЕ ВРЕМЯ НА СТРАНИЦУ: {df_clean['time_per_page'].mean():.2f} мин\")\n",
    "print(f\"📄 МЕДИАННОЕ ВРЕМЯ НА СТРАНИЦУ: {df_clean['time_per_page'].median():.2f} мин\")\n",
    "\n",
    "# Анализ по количеству страниц\n",
    "print(\"\\n📊 ВРЕМЯ ПО КОЛИЧЕСТВУ СТРАНИЦ:\")\n",
    "time_by_pages = df_clean.groupby('pages_count')['session_time_minutes'].agg(['mean', 'median', 'count']).round(2)\n",
    "time_by_pages.columns = ['Среднее время', 'Медианное время', 'Количество сессий']\n",
    "print(time_by_pages)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 💰 Анализ конверсии до Checkout"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Детальный анализ конверсии\n",
    "print(\"💰 ДЕТАЛЬНЫЙ АНАЛИЗ КОНВЕРСИИ ДО CHECKOUT\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "# Конверсия по источникам трафика\n",
    "print(\"📊 КОНВЕРСИЯ ПО ИСТОЧНИКАМ ТРАФИКА:\")\n",
    "conversion_by_source = df_clean.groupby('clean_referrer').agg({\n",
    "    'Display Name': 'count',\n",
    "    'has_checkout': 'sum',\n",
    "    'has_checkout': 'mean'\n",
    "}).round(3)\n",
    "\n",
    "conversion_by_source.columns = ['Всего сессий', 'Checkout сессии', 'Конверсия']\n",
    "conversion_by_source['Конверсия %'] = (conversion_by_source['Конверсия'] * 100).round(1)\n",
    "print(conversion_by_source)\n",
    "\n",
    "# Конверсия по длительности сессии\n",
    "print(\"\\n⏱️  КОНВЕРСИЯ ПО ДЛИТЕЛЬНОСТИ СЕССИИ:\")\n",
    "conversion_by_time = df_clean.groupby('session_category').agg({\n",
    "    'Display Name': 'count',\n",
    "    'has_checkout': 'sum',\n",
    "    'has_checkout': 'mean'\n",
    "}).round(3)\n",
    "\n",
    "conversion_by_time.columns = ['Всего сессий', 'Checkout сессии', 'Конверсия']\n",
    "conversion_by_time['Конверсия %'] = (conversion_by_time['Конверсия'] * 100).round(1)\n",
    "print(conversion_by_time)\n",
    "\n",
    "# Конверсия по количеству страниц\n",
    "print(\"\\n📄 КОНВЕРСИЯ ПО КОЛИЧЕСТВУ СТРАНИЦ:\")\n",
    "conversion_by_pages = df_clean.groupby('pages_count').agg({\n",
    "    'Display Name': 'count',\n",
    "    'has_checkout': 'sum',\n",
    "    'has_checkout': 'mean'\n",
    "}).round(3)\n",
    "\n",
    "conversion_by_pages.columns = ['Всего сессий', 'Checkout сессии', 'Конверсия']\n",
    "conversion_by_pages['Конверсия %'] = (conversion_by_pages['Конверсия'] * 100).round(1)\n",
    "print(conversion_by_pages)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Визуализация данных"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Создание графиков\n",
    "print(\"📊 СОЗДАНИЕ ВИЗУАЛИЗАЦИИ\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "# 1. Распределение времени сессий\n",
    "plt.figure(figsize=(12, 8))\n",
    "\n",
    "plt.subplot(2, 2, 1)\n",
    "plt.hist(df_clean['session_time_minutes'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')\n",
    "plt.title('Распределение времени сессий')\n",
    "plt.xlabel('Время (минуты)')\n",
    "plt.ylabel('Количество сессий')\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# 2. Распределение количества страниц\n",
    "plt.subplot(2, 2, 2)\n",
    "page_counts = df_clean['pages_count'].value_counts().sort_index()\n",
    "plt.bar(page_counts.index, page_counts.values, color='lightgreen', alpha=0.7)\n",
    "plt.title('Распределение количества страниц')\n",
    "plt.xlabel('Количество страниц')\n",
    "plt.ylabel('Количество сессий')\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# 3. Конверсия по сегментам\n",
    "plt.subplot(2, 2, 3)\n",
    "conversion_data = df_clean.groupby('session_category')['has_checkout'].mean() * 100\n",
    "plt.bar(conversion_data.index, conversion_data.values, color='gold', alpha=0.7)\n",
    "plt.title('Конверсия по сегментам времени')\n",
    "plt.xlabel('Сегмент')\n",
    "plt.ylabel('Конверсия (%)')\n",
    "plt.xticks(rotation=45)\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# 4. Источники трафика\n",
    "plt.subplot(2, 2, 4)\n",
    "traffic_sources = df_clean['clean_referrer'].value_counts().head(5)\n",
    "plt.pie(traffic_sources.values, labels=traffic_sources.index, autopct='%1.1f%%', startangle=90)\n",
    "plt.title('Источники трафика')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"✅ Графики созданы\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Интерактивные графики с Plotly\n",
    "print(\"🎨 СОЗДАНИЕ ИНТЕРАКТИВНЫХ ГРАФИКОВ\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "# 1. Воронка конверсии\n",
    "fig_funnel = go.Figure(go.Funnel(\n",
    "    y=['Главная страница', 'Услуги/Packages', 'Checkout'],\n",
    "    x=[funnel_steps['Главная страница'], funnel_steps['Услуги/Packages'], funnel_steps['Checkout']],\n",
    "    textinfo=\"value+percent initial\"\n",
    "))\n",
    "\n",
    "fig_funnel.update_layout(\n",
    "    title='Воронка конверсии сайта',\n",
    "    font=dict(size=14),\n",
    "    showlegend=False\n",
    ")\n",
    "\n",
    "fig_funnel.show()\n",
    "\n",
    "# 2. Время сессий по дням\n",
    "daily_stats = df_clean.groupby('session_date').agg({\n",
    "    'session_time_minutes': 'mean',\n",
    "    'Display Name': 'count',\n",
    "    'has_checkout': 'mean'\n",
    "}).reset_index()\n",
    "\n",
    "fig_daily = make_subplots(\n",
    "    rows=2, cols=1,\n",
    "    subplot_titles=('Среднее время сессии по дням', 'Количество сессий и конверсия по дням'),\n",
    "    vertical_spacing=0.1\n",
    ")\n",
    "\n",
    "fig_daily.add_trace(\n",
    "    go.Scatter(x=daily_stats['session_date'], y=daily_stats['session_time_minutes'], \n",
    "               mode='lines+markers', name='Время сессии'),\n",
    "    row=1, col=1\n",
    ")\n",
    "\n",
    "fig_daily.add_trace(\n",
    "    go.Bar(x=daily_stats['session_date'], y=daily_stats['Display Name'], \n",
    "           name='Количество сессий', yaxis='y'),\n",
    "    row=2, col=1\n",
    ")\n",
    "\n",
    "fig_daily.add_trace(\n",
    "    go.Scatter(x=daily_stats['session_date'], y=daily_stats['has_checkout']*100, \n",
    "               mode='lines+markers', name='Конверсия (%)', yaxis='y2'),\n",
    "    row=2, col=1\n",
    ")\n",
    "\n",
    "fig_daily.update_layout(\n",
    "    title='Динамика метрик по дням',\n",
    "    height=600,\n",
    "    showlegend=True\n",
    ")\n",
    "\n",
    "fig_daily.show()\n",
    "\n",
    "print(\"✅ Интерактивные графики созданы\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 Инсайты и рекомендации"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Анализ проблемных мест\n",
    "print(\"🔍 АНАЛИЗ ПРОБЛЕМНЫХ МЕСТ\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "# 1. Страницы с высоким bounce rate\n",
    "print(\"📊 СТРАНИЦЫ С ВЫСОКИМ BOUNCE RATE:\")\n",
    "starting_page_bounce = df_clean.groupby('Starting Page').agg({\n",
    "    'Display Name': 'count',\n",
    "    'pages_count': lambda x: (x == 1).sum()\n",
    "})\n",
    "\n",
    "starting_page_bounce['bounce_rate'] = (starting_page_bounce['pages_count'] / starting_page_bounce['Display Name'] * 100).round(1)\n",
    "starting_page_bounce.columns = ['Всего сессий', 'Bounce сессии', 'Bounce Rate (%)']\n",
    "starting_page_bounce = starting_page_bounce.sort_values('Bounce Rate (%)', ascending=False)\n",
    "\n",
    "print(starting_page_bounce)\n",
    "\n",
    "# 2. Анализ источников с низкой конверсией\n",
    "print(\"\\n📊 ИСТОЧНИКИ С НИЗКОЙ КОНВЕРСИЕЙ:\")\n",
    "low_conversion_sources = conversion_by_source[conversion_by_source['Конверсия %'] < 10]\n",
    "if not low_conversion_sources.empty:\n",
    "    print(low_conversion_sources)\n",
    "else:\n",
    "    print(\"Все источники показывают приемлемую конверсию (>10%)\")\n",
    "\n",
    "# 3. Анализ коротких сессий\n",
    "print(\"\\n⏱️  АНАЛИЗ КОРОТКИХ СЕССИЙ (< 1 минуты):\")\n",
    "short_sessions = df_clean[df_clean['session_time_minutes'] < 1]\n",
    "print(f\"Количество коротких сессий: {len(short_sessions)}\")\n",
    "print(f\"Процент от общего числа: {(len(short_sessions)/len(df_clean)*100):.1f}%\")\n",
    "\n",
    "if len(short_sessions) > 0:\n",
    "    print(\"\\nНачальные страницы коротких сессий:\")\n",
    "    short_session_pages = short_sessions['Starting Page'].value_counts()\n",
    "    for page, count in short_session_pages.head(5).items():\n",
    "        print(f\"- {page}: {page}: {count} раз\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Рекомендации по улучшению UX\n",
    "print(\"💡 РЕКОМЕНДАЦИИ ПО УЛУЧШЕНИЮ UX\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "print(\"🎯 ОСНОВНЫЕ РЕКОМЕНДАЦИИ:\")\n",
    "print(\"\"\"\n",
    "1. 📱 УЛУЧШЕНИЕ МОБИЛЬНОГО UX:\n",
    "   - Проверить адаптивность проблемных страниц\n",
    "   - Оптимизировать скорость загрузки\n",
    "   - Упростить навигацию для мобильных устройств\n",
    "\n",
    "2. 🛤️  ОПТИМИЗАЦИЯ ВОРОНКИ КОНВЕРСИИ:\n",
    "   - Добавить промежуточные страницы между услугами и checkout\n",
    "   - Улучшить CTA кнопки на ключевых страницах\n",
    "   - Добавить социальные доказательства (отзывы, сертификаты)\n",
    "\n",
    "3. 📄 УЛУЧШЕНИЕ СТРАНИЦ УСЛУГ:\n",
    "   - Добавить больше деталей о процессе обучения\n",
    "   - Показать преимущества школы\n",
    "   - Добавить FAQ секцию\n",
    "\n",
    "4. ⚡ ОПТИМИЗАЦИЯ СКОРОСТИ:\n",
    "   - Сжать изображения\n",
    "   - Минимизировать CSS/JS\n",
    "   - Использовать кэширование\n",
    "\n",
    "5. 🔍 A/B ТЕСТИРОВАНИЕ:\n",
    "   - Тестировать разные заголовки страниц\n",
    "   - Экспериментировать с расположением CTA\n",
    "   - Тестировать разные форматы контента\n",
    "\"\"\")\n",
    "\n",
    "print(\"\\n📊 КОНКРЕТНЫЕ ДЕЙСТВИЯ:\")\n",
    "print(\"\"\"\n",
    "1. Немедленно:\n",
    "   - Анализ страниц с высоким bounce rate\n",
    "   - Проверка скорости загрузки проблемных страниц\n",
    "\n",
    "2. В течение недели:\n",
    "   - Создание A/B тестов для ключевых страниц\n",
    "   - Оптимизация checkout процесса\n",
    "\n",
    "3. В течение месяца:\n",
    "   - Редизайн проблемных страниц\n",
    "   - Добавление новых элементов для удержания пользователей\n",
    "   - Внедрение аналитики для отслеживания улучшений\n",
    "\"\"\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📈 Заключение и следующие шаги"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Итоговый отчет\n",
    "print(\"📈 ИТОГОВЫЙ ОТЧЕТ ПО АНАЛИЗУ\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "print(f\"📊 ОБЩАЯ СТАТИСТИКА:\")\n",
    "print(f\"- Всего сессий: {total_sessions}\")\n",
    "print(f\"- Среднее время сессии: {avg_session_time:.2f} минут\")\n",
    "print(f\"- Bounce rate: {bounce_rate:.1f}%\")\n",
    "print(f\"- Конверсия в checkout: {checkout_conversion:.1f}%\")\n",
    "\n",
    "print(f\"\\n🎯 КЛЮЧЕВЫЕ ИНСАЙТЫ:\")\n",
    "print(f\"- {len(df_clean[df_clean['session_time_minutes'] < 1])} сессий длились менее 1 минуты\")\n",
    "print(f\"- {len(df_clean[df_clean['pages_count'] == 1])} пользователей покинули сайт после первой страницы\")\n",
    "print(f\"- {len(df_clean[df_clean['has_checkout'] == True])} пользователей дошли до checkout\")\n",
    "\n",
    "print(f\"\\n🚀 ПРИОРИТЕТНЫЕ ДЕЙСТВИЯ:\")\n",
    "print(\"1. Оптимизация страниц с высоким bounce rate\")\n",
    "print(\"2. Улучшение checkout процесса\")\n",
    "print(\"3. A/B тестирование ключевых элементов\")\n",
    "print(\"4. Оптимизация скорости загрузки\")\n",
    "\n",
    "print(f\"\\n📅 СЛЕДУЮЩИЕ ШАГИ:\")\n",
    "print(\"1. Внедрить рекомендации по UX\")\n",
    "print(\"2. Настроить дополнительную аналитику\")\n",
    "print(\"3. Провести повторный анализ через 2-4 недели\")\n",
    "print(\"4. Измерить улучшения в метриках\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "language": "python",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}